In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sys
sys.path.insert(0, '/content/gdrive/My Drive/Colab Notebooks')

In [3]:
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
from nst_utils import *
import numpy as np
import tensorflow as tf
import pprint
import imageio
%matplotlib inline

In [4]:
pp = pprint.PrettyPrinter(indent=4)
model = load_vgg_model("/content/gdrive/My Drive/Colab Notebooks/imagenet-vgg-verydeep-19.mat")
pp.pprint(model)


{   'avgpool1': <tf.Tensor 'AvgPool:0' shape=(1, 150, 200, 64) dtype=float32>,
    'avgpool2': <tf.Tensor 'AvgPool_1:0' shape=(1, 75, 100, 128) dtype=float32>,
    'avgpool3': <tf.Tensor 'AvgPool_2:0' shape=(1, 38, 50, 256) dtype=float32>,
    'avgpool4': <tf.Tensor 'AvgPool_3:0' shape=(1, 19, 25, 512) dtype=float32>,
    'avgpool5': <tf.Tensor 'AvgPool_4:0' shape=(1, 10, 13, 512) dtype=float32>,
    'conv1_1': <tf.Tensor 'Relu:0' shape=(1, 300, 400, 64) dtype=float32>,
    'conv1_2': <tf.Tensor 'Relu_1:0' shape=(1, 300, 400, 64) dtype=float32>,
    'conv2_1': <tf.Tensor 'Relu_2:0' shape=(1, 150, 200, 128) dtype=float32>,
    'conv2_2': <tf.Tensor 'Relu_3:0' shape=(1, 150, 200, 128) dtype=float32>,
    'conv3_1': <tf.Tensor 'Relu_4:0' shape=(1, 75, 100, 256) dtype=float32>,
    'conv3_2': <tf.Tensor 'Relu_5:0' shape=(1, 75, 100, 256) dtype=float32>,
    'conv3_3': <tf.Tensor 'Relu_6:0' shape=(1, 75, 100, 256) dtype=float32>,
    'conv3_4': <tf.Tensor 'Relu_7:0' shape=(1, 75, 100, 256)

In [0]:
#content_image = imageio.imread("/content/gdrive/My Drive/Colab Notebooks/my_content1.jpg")
#imshow(content_image);

In [0]:
def compute_content_cost(a_C, a_G):

    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    a_C_unrolled = tf.reshape(a_C,shape=[m,n_H*n_W,n_C])
    a_G_unrolled = tf.reshape(a_G,shape=[m,n_H*n_W,n_C])
    J_content = (1/(4*n_H*n_W*n_C))*(tf.reduce_sum(tf.square(tf.subtract(a_C_unrolled,a_G_unrolled))))
    return J_content

In [7]:
tf.reset_default_graph()

with tf.Session() as test:
    tf.set_random_seed(1)
    a_C = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    a_G = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    J_content = compute_content_cost(a_C, a_G)
    print("J_content = " + str(J_content.eval()))

J_content = 6.7655935


In [0]:
def gram_matrix(A):

    GA = tf.matmul(A,tf.transpose(A))

    
    return GA

In [9]:
tf.reset_default_graph()

with tf.Session() as test:
    tf.set_random_seed(1)
    A = tf.random_normal([3, 2*1], mean=1, stddev=4)
    GA = gram_matrix(A)
    
    print("GA = \n" + str(GA.eval()))

GA = 
[[ 6.422305 -4.429122 -2.096682]
 [-4.429122 19.465837 19.563871]
 [-2.096682 19.563871 20.686462]]


In [0]:
def compute_layer_style_cost(a_S, a_G):

    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    
    a_S = tf.reshape(tf.transpose(a_S,perm=[0,3,1,2]),shape=[n_C,n_H*n_W])
    a_G = tf.reshape(tf.transpose(a_G,perm=[0,3,1,2]),shape=[n_C,n_H*n_W])

    GS = tf.matmul(a_S,tf.transpose(a_S))
    GG = tf.matmul(a_G,tf.transpose(a_G))
    J_style_layer = (1/(4*(n_C**2)*(n_H*n_W)**2))*tf.reduce_sum(tf.square(tf.subtract(GS,GG)))
    
    
    return J_style_layer

In [11]:
tf.reset_default_graph()

with tf.Session() as test:
    tf.set_random_seed(1)
    a_S = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    a_G = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    J_style_layer = compute_layer_style_cost(a_S, a_G)
    
    print("J_style_layer = " + str(J_style_layer.eval()))

J_style_layer = 9.190277


In [0]:
STYLE_LAYERS = [
    ('conv1_1', 0.2),
    ('conv2_1', 0.2),
    ('conv3_1', 0.2),
    ('conv4_1', 0.2),
    ('conv5_1', 0.2)]

In [0]:
def compute_style_cost(model, STYLE_LAYERS):

    J_style = 0

    for layer_name, coeff in STYLE_LAYERS:


        out = model[layer_name]

        a_S = sess.run(out)

        a_G = out
  
        J_style_layer = compute_layer_style_cost(a_S, a_G)

        J_style += coeff * J_style_layer

    return J_style

In [0]:
def total_cost(J_content, J_style, alpha = 10, beta = 40):

    J = alpha*J_content + beta*J_style
    
    return J

In [15]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(3)
    J_content = np.random.randn()    
    J_style = np.random.randn()
    J = total_cost(J_content, J_style)
    print("J = " + str(J))

J = 35.34667875478276


In [0]:

tf.reset_default_graph()

sess = tf.InteractiveSession()

In [0]:
content_image = imageio.imread("/content/gdrive/My Drive/Colab Notebooks/mycont.jpg")
content_image = reshape_and_normalize_image(content_image)

In [0]:
style_image = imageio.imread("/content/gdrive/My Drive/Colab Notebooks/cubist.jpg")
style_image = reshape_and_normalize_image(style_image)

In [0]:
model = load_vgg_model("/content/gdrive/My Drive/Colab Notebooks/imagenet-vgg-verydeep-19.mat")

In [0]:

sess.run(model['input'].assign(content_image))

out = model['conv4_2']

a_C = sess.run(out)

a_G = out

J_content = compute_content_cost(a_C, a_G)

In [0]:

sess.run(model['input'].assign(style_image))

J_style = compute_style_cost(model, STYLE_LAYERS)

In [0]:
J = total_cost(J_content, J_style, alpha = 10, beta = 40)

In [0]:

optimizer = tf.train.AdamOptimizer(2.0)

train_step = optimizer.minimize(J)

In [0]:
def save_image1(path, image):
  
    image = image + CONFIG.MEANS
    image = np.clip(image[0], 0, 255).astype('uint8')
    imageio.imwrite(path, image)

In [0]:
def model_nn(sess, input_image, num_iterations = 1000):

    sess.run(tf.global_variables_initializer())


    sess.run(model["input"].assign(input_image))
 
    for i in range(num_iterations):
    
  
        sess.run(train_step)
  
        generated_image = sess.run(model['input'])

        if i%20 == 0:
            Jt, Jc, Js = sess.run([J, J_content, J_style])
            print("Iteration " + str(i) + " :")
            print("total cost = " + str(Jt))
            print("content cost = " + str(Jc))
            print("style cost = " + str(Js))
            
 
            save_image1("/content/gdrive/My Drive/Colab Notebooks/" + str(i) + ".png", generated_image)
    save_image1('/content/gdrive/My Drive/Colab Notebooks/generated_image.jpg', generated_image)
    
    return generated_image

In [27]:
model_nn(sess, generated_image)

Iteration 0 :
total cost = 13469407000.0
content cost = 10619.487
style cost = 336732500.0
Iteration 20 :
total cost = 2623837400.0
content cost = 18456.812
style cost = 65591324.0
Iteration 40 :
total cost = 1430319700.0
content cost = 19702.021
style cost = 35753068.0
Iteration 60 :
total cost = 1030769100.0
content cost = 20597.334
style cost = 25764078.0
Iteration 80 :
total cost = 826224100.0
content cost = 21392.14
style cost = 20650254.0
Iteration 100 :
total cost = 693961300.0
content cost = 22142.357
style cost = 17343496.0
Iteration 120 :
total cost = 597884740.0
content cost = 22830.395
style cost = 14941411.0
Iteration 140 :
total cost = 524873920.0
content cost = 23474.182
style cost = 13115979.0
Iteration 160 :
total cost = 467645900.0
content cost = 24060.734
style cost = 11685132.0
Iteration 180 :
total cost = 421609380.0
content cost = 24539.455
style cost = 10534099.0
Iteration 200 :
total cost = 384211420.0
content cost = 24947.404
style cost = 9599049.0
Iteration 22

array([[[[-5.1893330e+01,  9.2689600e+00,  9.3691193e+01],
         [-8.1951973e+01,  1.8004616e+01,  1.3064223e+02],
         [-6.8607552e+01,  1.6376127e+01,  1.0136405e+02],
         ...,
         [-2.8088264e+01,  2.4462511e+01,  3.5293190e+01],
         [-3.9430000e+01,  9.6914310e+00,  2.9162169e+01],
         [-4.3803978e+01,  1.1908092e+01,  9.8670158e+01]],

        [[-8.8396309e+01,  1.5476979e+01,  9.8429802e+01],
         [-3.7310024e+01,  1.2536489e+00,  6.1829647e+01],
         [-2.3048309e+01,  8.0092239e-01,  5.9574757e+01],
         ...,
         [-2.6018194e+01,  1.8225695e+01,  3.0345774e+01],
         [-2.9923531e+01,  5.7628887e+01,  5.8434391e+00],
         [-4.3567558e+01,  1.1690986e+01,  3.7657692e+01]],

        [[-7.7701492e+01,  1.7098490e+00,  8.1827522e+01],
         [-2.8366880e+01, -6.5800867e+00,  5.4357201e+01],
         [-2.5083218e+01,  9.9007406e+00,  4.3130692e+01],
         ...,
         [-9.0717506e+00, -1.2715713e+01,  7.9374771e+00],
         [